In [2]:
def main(Lum52_Lum88):
    """
    Estimate the electron density, Ne
    from ratio of luminosity of OIII 52 to OIII 88um lines, Lum52_Lum88
    using Einstein coefficients, effective collision strengths, and line frequencies.
    
    State 0 is 3P0 state with J = 0
    State 1 is 3P1; J = 1
    State 2 is 3P2; J = 2
    
    Transition from state 2 to 1, i.e., from 3P2 to 3P1 corresponds to the 52 um line
    Transition from state 1 to 0, i.e., 3P1 to 3P0 corresponds to the 88 um line
    
    """    
    import numpy as np
#    beta = ((2*np.pi*(c.hbar**4) / (c.k_B*(c.m_e**3)))**0.5).to(u.s**2*u.erg**1.5*u.K**0.5*u.g**-1.5)
    beta = 8.6291322e-6 # ((2 pi hbar^4) / (kB me^3)) ^ (1/2)
    sqrtT = np.sqrt(10000) # Assume temp ~ 10,000 K

# Einstein coefficients from NIST
    A10 = 2.61e-05 # s^-1 88.3564µm
    A20 = 3.17e-11 # s^-1 32.6612µm
    A21 = 9.76e-05 # s^-1 51.8145µm
    
    nu10 = 3393.0062440 # GHz # Line frequencies from Splatalogue
    nu21 = 5785.8795890 # GHz
    
    gammaO = np.array([[0.0000, 0.5421, 0.2568],  # Maxwellian averaged, aka effective collision strength
                       [0.5421, 0.0000, 1.2526],  # Storey+ MNRAS 441, 3028 (2014) Table 6 @ 10000K
                       [0.2568, 1.2526, 0.0000]]) # Assume symmetry of collision strengths


#     beta = 8.6e-6 # ((2 pi hbar^4) / (kB me^3)) ^ (1/2)
#     sqrtT = np.sqrt(10000) # Assume temp ~ 10,000 K

#     A10 = 2.7e-5 # s^-1 # Einstein coefficients from Osterbrock Table 3.12
#     A20 = 3.1e-11 # s^-1
#     A21 = 9.7e-5 # s^-1

#     nu10 = 3393.0 # Hz # Line frequencies
#     nu21 = 5785.9 # Hz

#     gammaO = np.array([[0, 0.55, 0.27], # Maxwellian averaged, aka effective collision strength
#                      [0.55 , 0, 1.29],  # Osterbrock Table 
#                      [0.27 , 1.29, 0]]) # Assume symmetry of collision strengths

    def gO(i): # Statistical weight
        return(2*i + 1)

    def qO(i,j): # Rate coefficient
        return(beta/sqrtT * gammaO[i,j]/gO(i))

    def Ne(Lum52_Lum88): # 52um/88um luminosity ratio (in Watts/Watts)
        
#         if (Lum52_Lum88 < 0) or (Lum52_Lum88 > 10.627):
#             rat = np.nan
        
#         else:

        n2_n1 = Lum52_Lum88 * (A10/A21) * (nu10/nu21) # ratio of electrons in state 2 / state 1

        num = A10/qO(0,1) - n2_n1 * ( (A21+A20)/qO(0,2) + A21/qO(0,1) )

        denom1 = n2_n1 * ( (qO(2,1)+qO(2,0))/qO(0,2) + qO(2,1)/qO(0,1) )
        denom2 = (qO(1,2)+qO(1,0))/qO(0,1) + qO(1,2)/qO(0,2)
        denom = denom1 - denom2

        rat = num/denom

        return(rat)
    
    return(Ne(Lum52_Lum88))